# Create Joint GeoJSON

---  
##### source tutorial: https://bost.ocks.org/mike/map/  
---

**You're starting with two files:**
* a _shape file_ of the world map
* a _csv file_ of your GPS coordinates

**We'll have to:**
* convert the _shape file_ to a _GeoJSON_ format
* convert the _csv file_ to a _GeoJSON_ format
* merge the two GeoJSON files into one data file
    * we'll merge into a _TopJSON file_ ... which is basically the same but much more compressed
    
**Note:**
* we'll be employing a couple of linux command-line tools here
    * ogr2ogr - creates a GeoJSON from shape files
    * topojson - creates a topoJSON from GeoJSON files
* in python notebooks, you can use "!" at the beginng of a command to run it in BASH SHELL
---

## World Map

* change directory to map folder

In [72]:
cd "../data/ne_10m_admin_0_map_subunits"

/Users/jlc/Google Drive/_code/cnc_map/data/ne_10m_admin_0_map_subunits


* check working directory

In [51]:
!pwd

/Users/jlc/Google Drive/_code/cnc_map/data/ne_10m_admin_0_map_subunits


* convert shape file to json

In [52]:
!ogr2ogr -f GeoJSON worldmap.json "ne_10m_admin_0_map_subunits.shp"

* check that file is present

In [63]:
!ls worldmap*

worldmap.json


* move GeoJSON to common folder

In [54]:
!mv worldmap.json "../worldmap.json"

* check moved correctly

In [82]:
!ls ../worldmap.json

../worldmap.json


---  
## CSV with Coordinates

* change directory to data folder

In [83]:
cd "../"

/Users/jlc/Google Drive/_code/cnc_map/data


* check working directory

In [84]:
!pwd

/Users/jlc/Google Drive/_code/cnc_map/data


* convert CSV to GeoJSON

In [57]:
import csv, json
from geojson import Feature, FeatureCollection, Point

def CSVtoGEOJSON (myCSV, myGeoJSON='myGeoJSON.json', charset='utf-8'):
    '''
    Takes a CSV file and converts it to GeoJSON format. Allows for 'charset' argument to set encoding.
    Assumes file structure, CSV with headers: place,longitude,latitude
    ''' 
    
    # generate GeoJSON format
    features = []                                                         # create empty list
    with open(myCSV, newline='', encoding=charset) as csvfile:            # open file
        reader = csv.reader(csvfile, delimiter=',')                       # read file in line-by-line
        next(reader)                                                      # skip header line
        for place, longitude, latitude in reader:                         # create dict arrays inside of list
            latitude, longitude = map(float, (latitude, longitude))
            features.append(
                Feature(
                    geometry = Point((longitude, latitude)),
                    properties = {
                        'name': place
                    }
                )
            )   
    
    # write file
    collection = FeatureCollection(features)                              # FeatureCollection function from GeoJSON
    with open(myGeoJSON, "w") as f:                                       # write file
        f.write('%s' % collection)
    
    # status message
    print (myCSV + '    converted to    ' + myGeoJSON)
    
CSVtoGEOJSON('MeteorStrikes.csv', 'MeteorStrikes.json', charset="iso-8859-1")

MeteorStrikes.csv    converted to    MeteorStrikes.json


* check file is present

In [86]:
!ls MeteorStrikes.json

MeteorStrikes.json


--- 
## Merge

* merge

In [88]:
!topojson -o worldmap_and_points.json --id-property SU_A3 --properties -- worldmap.json MeteorStrikes.json

bounds: -180 -90 180 83.63410065300008 (spherical)
pre-quantization: 40.0m (0.000360°) 19.3m (0.000174°)
topology: 4676 arcs, 481140 points
post-quantization: 4.003km (0.0360°) 1.931km (0.0174°)
prune: retained 4264 / 4676 arcs (91%)


* check file is present

In [89]:
!ls worldmap*

worldmap.json            worldmap_and_points.json
